# Run a REopt API evaluation

## Initialization

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import copy
import os
from src.post_and_poll import get_api_results
API_KEY = 'uiMp56xYMT983T2OikI0RrEKAyAAEDpZHzuC8nTC'  # REPLACE WITH YOUR API KEY

# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()
"""
Here are some convenience definitions for using the Multi-scenario capabilities
"""
##############################################################################################################
inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')
loads_path = os.path.join(".", 'load_profiles')
rates_path = os.path.join(".", 'electric_rates')
##############################################################################################################

## Load a previously saved API response .json file instead of running REopt

In [4]:
response_json = 'results'
with open(os.path.join(outputs_path, response_json + '.json'), 'rb') as handle:
    api_response = json.load(handle)

## Starting /ghpghx endpoint POST/inputs, to be appended to for ground conductivity and heating/cooling loads

In [14]:
ghpghx_post = {
    "borehole_depth_ft": 400.0,
    "borehole_spacing_ft": 20.0,
    "borehole_spacing_type": "rectangular",
    "max_eft_allowable_f": 104.0,
    "min_eft_allowable_f": 23.0,
    "heating_thermal_load_mmbtu_per_hr": [],  # Filled in below
    "cooling_thermal_load_ton": [],  # Filled in below
    # "ambient_temperature_f": [],  # Will call PVWatts API with Lat/Long to get temperature profile
    # "cop_map_eft_heating_cooling": [],  # Will use default if none provided
    "simulation_years": 25,
    "solver_eft_tolerance_f": 1.0,
    "ghx_model": "TESS",
    "tess_ghx_minimum_timesteps_per_hour": 1,
    "max_sizing_iterations": 12,
    "init_sizing_factor_ft_per_peak_ton": 500.0,
    "latitude": 37.7,  # Used for 
    "longitude": -105.0
}

### Call /ghpghx/ground_conductivity endpoint to get the default ground conductivity based on location

In [ ]:
ground_conductivity_url = root_url + '/ghpghx/ground_conductivity/?api_key=' + API_KEY 
data = {"latitude": ghpghx_post["latitude"],
        "longitude": ghpghx_post["longitude"]
       }
resp = requests.get(url=ground_conductivity_url, params=data, verify=False)
ground_conductivity_response = json.loads(resp.text)
ghpghx_post["ground_thermal_conductivity_btu_per_hr_ft_f"] = ground_conductivity_response['thermal_conductivity']

## TODO: call /simulated_load endpoint to get heating and cooling loads to post to /ghpghx endpoint below

In [27]:
# Placeholder heating and cooling load inputs
# Fuel-basis option for heating load
ghpghx_post["heating_fuel_load_mmbtu_per_hr"] = list(2 * np.ones(8760))
ghpghx_post["boiler_efficiency"] = 0.8
# Thermal-basis option for heating load
# ghpghx_post["heating_thermal_load_mmbtu_per_hr"] = list(2 * np.ones(8760))

# Thermal-basis for cooling load (only option)
ghpghx_post["cooling_thermal_load_ton"] = list(200 * np.ones(8760))

### This POST request below will actually run the full GHPGHX model before returning the "ghp_uuid" for getting the results, so this will take 1-2 minutes before finishing the execution of this cell

In [19]:
root_url = "https://developer.nrel.gov/api/reopt/v1"
post_url = root_url + '/ghpghx/?api_key=' + API_KEY
resp = requests.post(post_url, json=ghpghx_post)
post_resp = json.loads(resp.text)

### Get the results data from the /ghpghx endpoint using the ghp_uuid returned from the POST request

In [25]:
ghp_uuid = post_resp["ghp_uuid"]
results_url = root_url + '/ghpghx/' + ghp_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
ghpghx_response = json.loads(resp.text)

### Find the data you need (e.g. number of boreholes) from the response

In [37]:
ghpghx_response.keys()

dict_keys(['ghp_uuid', 'outputs', 'inputs', 'messages'])

In [38]:
ghpghx_response['outputs'].keys()

dict_keys(['number_of_boreholes', 'length_boreholes_ft', 'yearly_heating_heatpump_electric_consumption_series_kw', 'yearly_cooling_heatpump_electric_consumption_series_kw', 'yearly_ghx_pump_electric_consumption_series_kw', 'yearly_total_electric_consumption_series_kw', 'yearly_total_electric_consumption_kwh', 'peak_heating_heatpump_thermal_ton', 'peak_cooling_heatpump_thermal_ton', 'peak_combined_heatpump_thermal_ton', 'yearly_heat_pump_eft_series_f', 'max_eft_f', 'min_eft_f', 'heating_cop_avg', 'cooling_cop_avg', 'solved_eft_error_f'])

In [41]:
ghpghx_response["outputs"]["number_of_boreholes"]

610.0

In [39]:
ghpghx_response['inputs'].keys()

dict_keys(['latitude', 'longitude', 'borehole_depth_ft', 'ghx_header_depth_ft', 'borehole_spacing_ft', 'borehole_diameter_inch', 'borehole_spacing_type', 'ghx_pipe_outer_diameter_inch', 'ghx_pipe_wall_thickness_inch', 'ghx_pipe_thermal_conductivity_btu_per_hr_ft_f', 'ghx_shank_space_inch', 'ground_thermal_conductivity_btu_per_hr_ft_f', 'ground_mass_density_lb_per_ft3', 'ground_specific_heat_btu_per_lb_f', 'grout_thermal_conductivity_btu_per_hr_ft_f', 'ghx_fluid_specific_heat_btu_per_lb_f', 'ghx_fluid_mass_density_lb_per_ft3', 'ghx_fluid_thermal_conductivity_btu_per_hr_ft_f', 'ghx_fluid_dynamic_viscosity_lbm_per_ft_hr', 'ghx_fluid_flow_rate_gpm_per_ton', 'ghx_pump_power_watt_per_gpm', 'ghx_pump_min_speed_fraction', 'ghx_pump_power_exponent', 'max_eft_allowable_f', 'min_eft_allowable_f', 'heating_thermal_load_mmbtu_per_hr', 'heating_fuel_load_mmbtu_per_hr', 'existing_boiler_efficiency', 'cooling_thermal_load_ton', 'ambient_temperature_f', 'cop_map_eft_heating_cooling', 'simulation_years'

### Load in a custom electric rate generated from https://reopt.nrel.gov/tool/custom_tariffs

In [6]:
load_electric_rate = "PGE_E20"
with open(os.path.join(rates_path, load_electric_rate + ".json"), 'r') as fp:
    rate_1 = json.load(fp)
post_1["Scenario"]["Site"]["ElectricTariff"]["urdb_response"] = rate_1

## Save the POST to a .json file for sharing or future loading in

In [7]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
post_name = "post_1"
with open(os.path.join(inputs_path, post_name + ".json"), 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the inputs/POST

In [8]:
# Load a json into a python dictionary
load_post = "post_1"
with open(os.path.join(inputs_path, load_post + ".json"), 'r') as fp:
    post_1 = json.load(fp)

## POST and poll (periodic GET request) the API to GET a new result, if not loading in a previous response. This may take a while!

### Note, the `api_url` in the `get_api_results` function below calls the **production server** hosted API (master/main branch/version, publicly accessible)

#### For calling a locally-hosted (localhost) API, see:
- https://github.com/NREL/REopt_Lite_API/wiki/localhost-URLs-for-calling-locally-hosted-API

#### For calling an API hosted on an NREL-internal server (only NREL users can access this), see:
- https://github.nrel.gov/REopt/API_scripts/wiki/API-URLs-for-NREL-internal-servers

`get_api_results` POST's your inputs to the API `job` endpoint, which provides a `run_uuid` if the input is valid, and then polls the `results` endpoint using the `run_uuid` until the results come back with a status other than `Optimizing...`.

`get_api_results` also saves the results (full API response, including inputs) to the `results_file`.

A log file is also created in the current working directory.

In [9]:
outputs_file_name = "results_file"
root_url = "https://developer.nrel.gov/api/reopt/v1"
api_response = get_api_results(post=post_1, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file=os.path.join(outputs_path, outputs_file_name + ".json"), 
                               run_id=None)

main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=DEMO_KEY.
main         INFO     Polling https://developer.nrel.gov/api/reopt/v1/job/61f4822c-e213-4751-bf08-3f9faae184fb/results/?api_key=DEMO_KEY for results with interval of 5s...
main         INFO     Saved results to ./outputs/results_file.json


### If you get disconnected from the polling function but you think it ran, copy the run_uuid from the log above to manually GET the results:

In [10]:
run_uuid = api_response["outputs"]["Scenario"]["run_uuid"]
results_url = root_url + '/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

## Get summary of results

In [11]:
print("NPV ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["npv_us_dollars"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotTES", "ColdTES", "AbsorptionChiller", "GHP", "NewBoiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1["Scenario"]["Site"].keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        # PV and Storage are considered if the POST does not explicitly make max_[size] == 0
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])
    elif tech in ["PV", "Storage"]:
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])                

NPV ($) =  550779.0
Capital Cost, Net ($) =  294265.01
PV size_kw  =  216.6667
Storage size_kw  =  96.4996266256644
Storage size_kwh  =  273.818171101346


### Here are some results keys examples:

In [12]:
api_response.keys()

dict_keys(['outputs', 'inputs', 'messages'])

In [14]:
api_response["outputs"]["Scenario"]["status"]

'optimal'

In [16]:
for k in api_response["outputs"]["Scenario"]["Site"].keys():
    print(k)

year_one_emissions_lb_C02
year_one_emissions_bau_lb_C02
outdoor_air_temp_degF
renewable_electricity_energy_pct
Financial
LoadProfile
LoadProfileBoilerFuel
LoadProfileChillerThermal
ElectricTariff
FuelTariff
Storage
Generator
Wind
CHP
Boiler
ElectricChiller
AbsorptionChiller
HotTES
ColdTES
NewBoiler
SteamTurbine
GHP
PV


## Save API response into a JSON file for later use

In [17]:
response_save = api_response
file_name_to_save = "response_1"
with open(os.path.join(outputs_path, file_name_to_save + ".json"), 'w') as fp:
    json.dump(response_save, fp)